In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# read csv file and uplooad it as a data frame
weather_df = pd.read_csv('weatherAPI_data.csv')

#weather_df.head()